# Download da base de dados do Supabase

In [ ]:
from dotenv import load_dotenv
import pandas as pd
import os
from supabase import create_client



load_dotenv()

SUPABASE_URL = os.environ.get("SUPABASE_URL")
SUPABASE_KEY = os.environ.get("SUPABASE_KEY")
pd.set_option('display.max_columns', None)

supabase = create_client(SUPABASE_URL, SUPABASE_KEY)
query = supabase.table("wine_data").select("*").execute()

In [66]:
db_original = pd.DataFrame(query.data)
db = db_original.copy()

# Informações básica do DataFrame

In [67]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 623 entries, 0 to 622
Data columns (total 32 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   id                                   623 non-null    int64 
 1   product_type                         584 non-null    object
 2   product_name                         623 non-null    object
 3   wine_variety                         584 non-null    object
 4   wine_region                          623 non-null    object
 5   item_quantity                        623 non-null    object
 6   wine_grapes                          281 non-null    object
 7   color_description                    421 non-null    object
 8   scent_description                    420 non-null    object
 9   taste_description                    422 non-null    object
 10  fruit_tasting                        422 non-null    object
 11  sugar_tasting                        422 non-

In [68]:
db.describe()

,id
count,623.000000
mean,312.410915
std,180.671981
min,1.000000
25%,156.500000
50%,312.000000
75%,467.500000
max,631.000000


# Feature Engineering

## Feature: Item quantity -> int 

### Resultado: Foi criada uma feature com o numero de unidade com o tipo int o nome da coluna é: item_quantity_integer

In [69]:
teste = db['item_quantity'].sort_values(ascending=False, key=lambda x: x.str.len())
teste

14     10 unidades
25     10 unidades
573    10 unidades
268    10 unidades
277    10 unidades
          ...     
296              1
295              1
292              1
290              1
287              1
Name: item_quantity, Length: 623, dtype: object

In [70]:
db['item_quantity_integer'] = db['item_quantity'].sort_values(ascending=False, key=lambda x: x.str.len())
db['item_quantity_integer'] = db['item_quantity'].str.split(" ").apply(lambda x: x[0]).astype(int)
db['item_quantity_integer']

0      1
1      1
2      6
3      4
4      6
      ..
618    3
619    1
620    3
621    1
622    1
Name: item_quantity_integer, Length: 623, dtype: int64

In [71]:
db['item_quantity_integer'].max()

np.int64(12)

## Feature: wine_variety -> Retirando os product type nulos. Não são vinhos

### Resultado: As entradas foram retiradas do dataframe que irá pro modelo por possuir uma variedade de itens que não são vinhos

In [ ]:
db["wine_variety"].unique()

array(['Vinho branco sul-africano', 'Vinho tinto argentino', 'Kit',
       'Vinho tinto italiano', 'Vinho tinto chileno',
       'Vinho tinto português', 'Vinho tinto europeu',
       'Vinho rosé europeu', 'Vinho tinto sul-africano',
       'Espumante Branco brasileiro', None, 'Vinho tinto espanhol',
       'Vinho rosé chileno', 'Vinho tinto francês',
       'Vinho rosé argentino', 'Vinho branco chileno',
       'Espumante Rosé espanhol', 'Espumante Branco espanhol',
       'Vinho rosé sul-africano', 'Vinho rosé espanhol',
       'Vinho branco argentino', 'Vinho branco italiano',
       'Vinho branco português', 'Espumante Rosé brasileiro',
       'Vinho branco francês', 'Vinho branco espanhol',
       'Espumante Branco francês', 'Vinho rosé português',
       'Vinho rosé italiano', 'Espumante Rosé francês',
       'Espumante Branco argentino', 'Vinho rosé francês',
       'Vinho branco brasileiro'], dtype=object)

In [73]:
db["wine_variety"].unique()

array(['Vinho branco', 'Vinho tinto', 'Vários tipos', 'Vinho rosé',
       'Espumante Branco', None, 'Espumante Rosé'], dtype=object)

In [77]:
db[db['wine_variety'].isna()].head()

,id,product_type,product_name,wine_variety,wine_region,item_quantity,wine_grapes,color_description,scent_description,taste_description,...,technical_sheet_producer,technical_sheet_crop_year,technical_sheet_cellaring_time,technical_sheet_maturation_time,created_at,specialist_review_content,specialist_review_owner,specialist_review_occupation,photo_url,item_quantity_integer


In [76]:
db = db[db["wine_variety"].isna()==False]

## Feature: product_type 

### Resultado: Os kits são conjuntos de produtos cujas páginas da Evino possuem muito menos informação, o que não é bom para o modelo. Desta forma, foi necessário retirar e criar um dataframe a parte para caso queiramos utilizar mais a frente.

In [79]:
db[db["product_type"]=="Kit"]

,id,product_type,product_name,wine_variety,wine_region,item_quantity,wine_grapes,color_description,scent_description,taste_description,...,technical_sheet_producer,technical_sheet_crop_year,technical_sheet_cellaring_time,technical_sheet_maturation_time,created_at,specialist_review_content,specialist_review_owner,specialist_review_occupation,photo_url,item_quantity_integer
2,11,Kit,Kit Primitivos Best Sellers | 6 garrafas por R...,Vinho tinto,Itália,6 unidades,None,None,None,None,...,None,None,None,None,2025-03-24T02:37:57.406997,None,None,None,None,6
3,1,Kit,Kit 3 Malbecs Best Sellers + Bolsa Térmica,Vários tipos,Vários países,4 unidades,None,None,None,None,...,None,None,None,None,2025-03-24T03:18:12.856299,None,None,None,None,4
4,4,Kit,Kit Linha Vini de Angeli | 2 Primitivo + 2 Pin...,Vários tipos,Itália,6 unidades,None,None,None,None,...,None,None,None,None,2025-03-24T03:18:27.095948,None,None,None,None,6
8,16,Kit,Kit Première Bulle | 2 Branco + 2 Rosé por R$1...,Vários tipos,França,4 unidades,None,None,None,None,...,None,None,None,None,2025-03-23T17:56:44.828058,None,None,None,None,4
11,18,Kit,Kit Vinhos Premiados Best Sellers | 5 garrafas...,Vinho tinto,Vários países,5 unidades,None,None,None,None,...,None,None,None,None,2025-03-23T17:57:23.282315,None,None,None,None,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,447,Kit,Kit Linha Atardecer de Los Andes + Kit 6 Marca...,Vários tipos,Vários países,5 unidades,None,None,None,None,...,None,None,None,None,2025-03-24T00:55:01.854817,None,None,None,None,5
573,550,Kit,"Kit 10 Vinhos Tintos por R$22,90 cada garrafa*",Vinho tinto,Vários países,10 unidades,None,None,None,None,...,None,None,None,None,2025-03-24T01:59:49.594106,None,None,None,None,10
594,25,Kit,"Kit 3 Espumantes Best Sellers por R$109,90 cad...",Vários tipos,Vários países,3 unidades,None,None,None,None,...,None,None,None,None,2025-03-24T02:38:13.208747,None,None,None,None,3
600,102,Kit,Kit Primitivos di Manduria | 2 Casale Brondell...,Vinho tinto,Itália,4 unidades,None,None,None,None,...,None,None,None,None,2025-03-24T02:41:25.57598,None,None,None,None,4


In [80]:
db_kits = db[db['product_type']=="Kit"]
db = db[db['product_type']!="Kit"]

In [82]:
db.info()

<class 'pandas.core.frame.DataFrame'>
Index: 431 entries, 0 to 622
Data columns (total 33 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   id                                   431 non-null    int64 
 1   product_type                         431 non-null    object
 2   product_name                         431 non-null    object
 3   wine_variety                         431 non-null    object
 4   wine_region                          431 non-null    object
 5   item_quantity                        431 non-null    object
 6   wine_grapes                          281 non-null    object
 7   color_description                    421 non-null    object
 8   scent_description                    420 non-null    object
 9   taste_description                    422 non-null    object
 10  fruit_tasting                        422 non-null    object
 11  sugar_tasting                        422 non-null 

## Feature wine_grapes -> Avaliação

### Resultado: Talvez seja improtante retirar essas entradas do dataframe principal, assim como fiz com os kits e separá-las num outro df que juntarei com o df dos kits. 

In [83]:
db["wine_grapes"].unique()

array(['Sauvignon Blanc', 'Malbec', None, 'Uvas variadas',
       'Cabernet Sauvignon', 'Jaén, Touriga Nacional, Aragonez', 'Blend',
       'Tempranillo, Merlot, Syrah', 'Carménère', 'Montepulciano',
       'Tempranillo', 'Merlot', 'Torrontés', 'Pinot Grigio',
       'Arinto, Trajadura', 'Chardonnay', 'Garnacha, Syrah, Tempranillo',
       'Pinot Noir', 'Bobal', 'Primitivo', 'Blend, Tempranillo',
       'Garnacha, Tempranillo', 'Nebbiolo', 'Garnacha',
       'Grenache, Syrah, Tempranillo',
       'Blend, Chardonnay, Chenin Blanc, Torrontés', 'Syrah', 'Airén',
       'Barbera, Dolcetto, Freisa, Bonarda, Albarossa, Merlot, Syrah, Cabernet Sauvignon',
       'Blend, Negroamaro, Primitivo, Uvas variadas', 'Moscatel',
       'Touriga Franca, Tinta Roriz', 'Blend, Seara Nova, Vital',
       'Bobal, Tempranillo',
       'Monastrell, Cabernet Sauvignon, Tempranillo', 'Sangiovese',
       'Merlot, Cabernet Sauvignon', 'Grenache, Mourvèdre, Syrah',
       'Merlot, Cabernet Sauvignon, Primitivo, 

In [86]:
db[db["wine_grapes"].isna()]

,id,product_type,product_name,wine_variety,wine_region,item_quantity,wine_grapes,color_description,scent_description,taste_description,...,technical_sheet_producer,technical_sheet_crop_year,technical_sheet_cellaring_time,technical_sheet_maturation_time,created_at,specialist_review_content,specialist_review_owner,specialist_review_occupation,photo_url,item_quantity_integer
5,5,Vinho tinto italiano,Kit 6 C.O.S 1960 Primitivo di Manduria DOP por...,Vinho tinto,Itália,6 unidades,None,Vermelho-rubi intenso com reflexos violáceos,"Intenso, com notas de frutas vermelhas e preta...","Rico, encorpado, sedoso e frutado, com taninos...",...,Cantolio,2022,2025,None,2025-03-24T03:18:40.820898,None,Vinícius Santiago,Sommelier da evino,None,6
7,10,Vinho tinto chileno,Kit 6 Reyna Cabernet Sauvignon Central Valley ...,Vinho tinto,Chile,6 unidades,None,Vermelho-rubi intenso,"Notas intensas de figos maduros e azeitonas, c...","Rico, macio e elegante",...,Bodega Tagua Tagua,2023,2025,None,2025-03-24T03:19:08.864529,None,Vinícius Santiago,Sommelier da evino,None,6
20,40,Vinho tinto sul-africano,Kit 3 The African Horizon Pinotage Western Cap...,Vinho tinto,África do Sul,3 unidades,None,Vermelho-rubi,"Notas de frutas vermelhas, como cereja e grose...","Frutado e suculento, ressaltando taninos macio...",...,Origin Wine,2023,2025,None,2025-03-23T18:03:31.029735,None,Vinícius Santiago,Sommelier da evino,https://res.cloudinary.com/evino/image/upload/...,3
22,44,Espumante Branco brasileiro,"Kit 6 Alísios Brut por R$74,90 cada garrafa",Espumante Branco,Brasil,6 unidades,None,Amarelo pálido com reflexos esverdeados,"Aromas intensos de frutas tropicais e flores, ...","Equilibrado, refrescante e agradável, com perl...",...,Miolo Wine Group,None,2025,None,2025-03-23T18:04:43.75337,None,Ari Gorenstein,Sommelier da evino,https://res.cloudinary.com/evino/image/upload/...,6
28,49,Vinho tinto espanhol,Kit 6 Valtier Reserva Utiel-Requena DOP por R$...,Vinho tinto,Espanha,6 unidades,None,Vermelho-rubi com reflexos violáceos,"Ameixa madura, pimenta, tostado e especiarias","Encorpado, equilibrado, taninos macios e redondos",...,Marqués del Atrio,2018,2025,Estágio em barricas de carvalho americano,2025-03-23T18:06:16.595191,None,Vinícius Santiago,Sommelier da evino,https://res.cloudinary.com/evino/image/upload/...,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,285,Vinho tinto italiano,Kit 4 Infinitum Montepulciano d'Abruzzo DOC po...,Vinho tinto,Itália,4 unidades,None,Vermelho-rubi,Notas de cerejas maduras e amoras,"Equilibrado e encorpado, com taninos macios",...,Deangeli Musti Nobilis,None,2025,None,2025-03-24T03:20:25.177931,None,Vinícius Santiago,Sommelier da evino,None,4
616,306,Vinho tinto italiano,Kit 3 Conte Parelli Appassimento Puglia IGT po...,Vinho tinto,Itália,3 unidades,None,Vermelho-rubi intenso com reflexos violáceos,"Notas intensas de frutas vermelhas maduras, e ...","Elegante e harmonioso, com taninos aveludados ...",...,Angelo Rocca e Figli Srl,2021,2025,Estágio em barricas de carvalho francês,2025-03-24T03:21:06.626014,None,Vinícius Santiago,Sommelier da evino,None,3
617,354,Espumante Branco francês,Kit 4 Première Bulle Blanquette de Limoux AOC ...,Espumante Branco,França,4 unidades,None,Dourado pálido com perlages finos,Aromas delicados de frutas cítricas e flores b...,"Vivaz e redondo, com textura agradável, notas ...",...,Sieur d'Arques,None,2025,Envelhecimento sobre as borras por 18 meses,2025-03-24T03:22:08.758223,None,Vinícius Santiago,Sommelier da evino,None,4
618,383,Vinho branco sul-africano,Kit 3 The African Horizon Chenin Blanc Western...,Vinho branco,África do Sul,3 unidades,None,Amarelo-palha,"Aromas frescos de frutas cítricas, de caroço e...","Leve e refrescante, com bom equilíbrio entre f...",...,Origin Wine,2023,2025,None,2025-03-24T03:22:51.982845,None,Vinícius Santiago,Sommelier da evino,None,3


In [88]:
db["product_name_with_word_kit"] = db['product_name'].str.contains("Kit", case=False)

In [95]:
db[db["product_name_with_word_kit"]==True].info()

<class 'pandas.core.frame.DataFrame'>
Index: 150 entries, 5 to 620
Data columns (total 34 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   id                                   150 non-null    int64 
 1   product_type                         150 non-null    object
 2   product_name                         150 non-null    object
 3   wine_variety                         150 non-null    object
 4   wine_region                          150 non-null    object
 5   item_quantity                        150 non-null    object
 6   wine_grapes                          0 non-null      object
 7   color_description                    148 non-null    object
 8   scent_description                    147 non-null    object
 9   taste_description                    148 non-null    object
 10  fruit_tasting                        148 non-null    object
 11  sugar_tasting                        148 non-null 

In [97]:
pd.set_option('display.max_columns', None)
db[db["product_name_with_word_kit"]==True]

,id,product_type,product_name,wine_variety,wine_region,item_quantity,wine_grapes,color_description,scent_description,taste_description,fruit_tasting,sugar_tasting,acidity_tasting,tannin_tasting,harmonizes_with,technical_sheet_wine_type,technical_sheet_volume,technical_sheet_closure_type,technical_sheet_service_temperature,technical_sheet_country,technical_sheet_region,technical_sheet_alcohol_content,technical_sheet_grapes,technical_sheet_producer,technical_sheet_crop_year,technical_sheet_cellaring_time,technical_sheet_maturation_time,created_at,specialist_review_content,specialist_review_owner,specialist_review_occupation,photo_url,item_quantity_integer,product_name_with_word_kit
5,5,Vinho tinto italiano,Kit 6 C.O.S 1960 Primitivo di Manduria DOP por...,Vinho tinto,Itália,6 unidades,None,Vermelho-rubi intenso com reflexos violáceos,"Intenso, com notas de frutas vermelhas e preta...","Rico, encorpado, sedoso e frutado, com taninos...",4,1,3,3,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Rolha de cortiça,18ºC,Itália,Puglia,14%,Primitivo,Cantolio,2022,2025,None,2025-03-24T03:18:40.820898,None,Vinícius Santiago,Sommelier da evino,None,6,True
7,10,Vinho tinto chileno,Kit 6 Reyna Cabernet Sauvignon Central Valley ...,Vinho tinto,Chile,6 unidades,None,Vermelho-rubi intenso,"Notas intensas de figos maduros e azeitonas, c...","Rico, macio e elegante",4,1,2,3,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Rolha de cortiça,18ºC,Chile,Valle Central,12.5%,Cabernet Sauvignon,Bodega Tagua Tagua,2023,2025,None,2025-03-24T03:19:08.864529,None,Vinícius Santiago,Sommelier da evino,None,6,True
20,40,Vinho tinto sul-africano,Kit 3 The African Horizon Pinotage Western Cap...,Vinho tinto,África do Sul,3 unidades,None,Vermelho-rubi,"Notas de frutas vermelhas, como cereja e grose...","Frutado e suculento, ressaltando taninos macio...",3,1,3,3,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Tampa de rosca,15ºC,África do Sul,Western Cape,13%,Pinotage,Origin Wine,2023,2025,None,2025-03-23T18:03:31.029735,None,Vinícius Santiago,Sommelier da evino,https://res.cloudinary.com/evino/image/upload/...,3,True
22,44,Espumante Branco brasileiro,"Kit 6 Alísios Brut por R$74,90 cada garrafa",Espumante Branco,Brasil,6 unidades,None,Amarelo pálido com reflexos esverdeados,"Aromas intensos de frutas tropicais e flores, ...","Equilibrado, refrescante e agradável, com perl...",4,1,4,1,"Carnes brancas, Frutos do mar, Queijos, Risoto...",Espumante Branco,750ml,Rolha de cortiça,8ºC,Brasil,Vale do São Francisco,12%,"Verdejo, Sauvignon Blanc, Chenin Blanc",Miolo Wine Group,None,2025,None,2025-03-23T18:04:43.75337,None,Ari Gorenstein,Sommelier da evino,https://res.cloudinary.com/evino/image/upload/...,6,True
28,49,Vinho tinto espanhol,Kit 6 Valtier Reserva Utiel-Requena DOP por R$...,Vinho tinto,Espanha,6 unidades,None,Vermelho-rubi com reflexos violáceos,"Ameixa madura, pimenta, tostado e especiarias","Encorpado, equilibrado, taninos macios e redondos",4,1,3,3,"Carnes de caça, Carnes vermelhas, Pizzas e mas...",Vinho tinto,750ml,Rolha de cortiça,16ºC,Espanha,Utiel-Requena,13%,"Tempranillo, Bobal",Marqués del Atrio,2018,2025,Estágio em barricas de carvalho americano,2025-03-23T18:06:16.595191,None,Vinícius Santiago,Sommelier da evino,https://res.cloudinary.com/evino/image/upload/...,6,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,285,Vinho tinto italiano,Kit 4 Infinitum Montepulciano d'Abruzzo DOC po...,Vinho tinto,Itália,4 unidades,None,Vermelho-rubi,Notas de cerejas maduras e amoras,"Equilibrado e encorpado, com taninos macios",3,1,3,2,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Rolha,16ºC,Itália,Abruzzo,12.5%,Montepulciano,Deangeli Musti Nobilis,None,2025,None,2025-03-24T03:20:25.177931,None,Vinícius Santiago,Sommelier da evino,None,4,True
616,306,Vinho tinto italiano,Kit 3 Cont

In [98]:
db[db["product_name"].str.contains("African Horizon Pinotage Western")]

,id,product_type,product_name,wine_variety,wine_region,item_quantity,wine_grapes,color_description,scent_description,taste_description,fruit_tasting,sugar_tasting,acidity_tasting,tannin_tasting,harmonizes_with,technical_sheet_wine_type,technical_sheet_volume,technical_sheet_closure_type,technical_sheet_service_temperature,technical_sheet_country,technical_sheet_region,technical_sheet_alcohol_content,technical_sheet_grapes,technical_sheet_producer,technical_sheet_crop_year,technical_sheet_cellaring_time,technical_sheet_maturation_time,created_at,specialist_review_content,specialist_review_owner,specialist_review_occupation,photo_url,item_quantity_integer,product_name_with_word_kit
20,40,Vinho tinto sul-africano,Kit 3 The African Horizon Pinotage Western Cap...,Vinho tinto,África do Sul,3 unidades,None,Vermelho-rubi,"Notas de frutas vermelhas, como cereja e grose...","Frutado e suculento, ressaltando taninos macio...",3,1,3,3,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Tampa de rosca,15ºC,África do Sul,Western Cape,13%,Pinotage,Origin Wine,2023,2025,None,2025-03-23T18:03:31.029735,None,Vinícius Santiago,Sommelier da evino,https://res.cloudinary.com/evino/image/upload/...,3,True
570,543,Vinho tinto sul-africano,The African Horizon Pinotage Western Cape W.O....,Vinho tinto,África do Sul•,1,Pinotage,Vermelho-rubi,"Notas de frutas vermelhas, como cereja e grose...","Frutado e suculento, ressaltando taninos macio...",3,1,3,3,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Tampa de rosca,15ºC,África do Sul,Western Cape,13%,Pinotage,Origin Wine,2023,2025,None,2025-03-24T01:58:16.95541,"Uva tinta mais famosa da África do Sul, a Pino...",Vinícius Santiago,Sommelier da evino,None,1,False


Observação: 
Como as entradas são identicas mas com menos volume de informações, por exemplo: em kits de produtos tem as avaliações de força de acidez, nível de fruta e etc mas não tem a avaliação do especialista. Talvez seja improtante retirar essas entradas do dataframe principal, assim como fiz com os kits e separá-las num outro df que juntarei com o df dos kits.

In [99]:
db_kits_from_product_name = db[db['product_name_with_word_kit']==True]
db = db[db['product_name_with_word_kit']==False]


db_all_kits = pd.concat([db_kits, db_kits_from_product_name], axis=0,ignore_index=True)

In [100]:
db_all_kits

,id,product_type,product_name,wine_variety,wine_region,item_quantity,wine_grapes,color_description,scent_description,taste_description,fruit_tasting,sugar_tasting,acidity_tasting,tannin_tasting,harmonizes_with,technical_sheet_wine_type,technical_sheet_volume,technical_sheet_closure_type,technical_sheet_service_temperature,technical_sheet_country,technical_sheet_region,technical_sheet_alcohol_content,technical_sheet_grapes,technical_sheet_producer,technical_sheet_crop_year,technical_sheet_cellaring_time,technical_sheet_maturation_time,created_at,specialist_review_content,specialist_review_owner,specialist_review_occupation,photo_url,item_quantity_integer,product_name_with_word_kit
0,11,Kit,Kit Primitivos Best Sellers | 6 garrafas por R...,Vinho tinto,Itália,6 unidades,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-03-24T02:37:57.406997,None,None,None,None,6,NaN
1,1,Kit,Kit 3 Malbecs Best Sellers + Bolsa Térmica,Vários tipos,Vários países,4 unidades,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-03-24T03:18:12.856299,None,None,None,None,4,NaN
2,4,Kit,Kit Linha Vini de Angeli | 2 Primitivo + 2 Pin...,Vários tipos,Itália,6 unidades,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-03-24T03:18:27.095948,None,None,None,None,6,NaN
3,16,Kit,Kit Première Bulle | 2 Branco + 2 Rosé por R$1...,Vários tipos,França,4 unidades,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-03-23T17:56:44.828058,None,None,None,None,4,NaN
4,18,Kit,Kit Vinhos Premiados Best Sellers | 5 garrafas...,Vinho tinto,Vários países,5 unidades,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2025-03-23T17:57:23.282315,None,None,None,None,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,285,Vinho tinto italiano,Kit 4 Infinitum Montepulciano d'Abruzzo DOC po...,Vinho tinto,Itália,4 unidades,None,Vermelho-rubi,Notas de cerejas maduras e amoras,"Equilibrado e encorpado, com taninos macios",3,1,3,2,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Rolha,16ºC,Itália,Abruzzo,12.5%,Montepulciano,Deangeli Musti Nobilis,None,2025,None,2025-03-24T03:20:25.177931,None,Vinícius Santiago,Sommelier da evino,None,4,True
299,306,Vinho tinto italiano,Kit 3 Conte Parelli Appassimento Puglia IGT po...,Vinho tinto,Itália,3 unidades,None,Vermelho-rubi intenso com reflexos violáceos,"Notas intensas de frutas vermelhas maduras, e ...","Elegante e harmonioso, com taninos aveludados ...",4,2,3,3,"Carnes vermelhas, Pizzas e massas de molho ver...",Vinho tinto,750ml,Rolha de cortiça,17ºC,Itália,Puglia,15%,"Blend, Negroamaro, Uvas variadas",Angelo Rocca e Figli Srl,2021,2025,Estágio em barricas de carvalho francês,2025-03-24T03:21:06.626014,None,Vinícius Santiago,Sommelier da evino,None,3,True
300,354,Espumante Branco francês,Kit 4 Première Bulle Blanquette de Limoux AOC ...,Espumante Branco,França,4 unidades,None,Dourado pálido com perlages finos,Aromas delicados de frutas cítricas e flores b...,"Vivaz e redondo, com textura agradável, notas ...",3,1,4,1,"Carnes brancas, Frutos do mar, Risoto e massas...",Espumante Branco,750ml,Rolha de cortiça,6ºC,França,Languedoc-Roussillon,12.5%,"Mauzac, Chardonnay, Chenin Blanc",Sieur d'Arques,None,2025,Envelhecimento sobre as borras por 18 meses,2025-03-24T03:22:08.758223,None,Vinícius Santiago,Sommelier da evino,None,4,True
301,383,Vinho branco sul-africano,Kit 3 The African Horizon Chenin Blanc Western...,Vinho branco,África do Sul,3 unidades,None,Amarelo-palha,"Aromas frescos de frutas cítricas, de caroço e...","Leve e refrescante, com bom equilíbrio entre f...",3,1,4,1,"Carnes brancas, Frutos do mar, Queijos, Salada...",Vinho branco,750ml

In [102]:
db.info()

<class 'pandas.core.frame.DataFrame'>
Index: 281 entries, 0 to 622
Data columns (total 34 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   id                                   281 non-null    int64 
 1   product_type                         281 non-null    object
 2   product_name                         281 non-null    object
 3   wine_variety                         281 non-null    object
 4   wine_region                          281 non-null    object
 5   item_quantity                        281 non-null    object
 6   wine_grapes                          281 non-null    object
 7   color_description                    273 non-null    object
 8   scent_description                    273 non-null    object
 9   taste_description                    274 non-null    object
 10  fruit_tasting                        274 non-null    object
 11  sugar_tasting                        274 non-null 

## Features: wine_grapes e technical_sheet_grapes

### Resultado: Podemos desconsiderar uma das duas colunas dado que os valores são todos iguais.

In [118]:
db[['wine_grapes', "technical_sheet_grapes"]]

,wine_grapes,technical_sheet_grapes
0,Sauvignon Blanc,Sauvignon Blanc
1,Malbec,Malbec
6,Uvas variadas,Uvas variadas
9,Cabernet Sauvignon,Cabernet Sauvignon
10,Uvas variadas,Uvas variadas
...,...,...
613,Malbec,Malbec
614,Grenache,Grenache
619,Nebbiolo,Nebbiolo
621,Blend,Blend


In [119]:
db['wine_grapes_equal_tech_sheet'] = db['wine_grapes']==db['technical_sheet_grapes']
db['wine_grapes_equal_tech_sheet'].value_counts()

C:\Users\victo\AppData\Local\Temp\ipykernel_30328\19708048.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db['wine_grapes_equal_tech_sheet'] = db['wine_grapes']==db['technical_sheet_grapes']


wine_grapes_equal_tech_sheet
True    281
Name: count, dtype: int64

Podemos desconsiderar uma das duas colunas dado que os valores são todos iguais.

## Features: wine_variety e technical_sheet_wine_type

### Resultado: A variável product_type é uma informação mais detalhada das outras duas variáveis

In [122]:
db[["product_type", "wine_variety", "technical_sheet_wine_type"]]

,product_type,wine_variety,technical_sheet_wine_type
0,Vinho branco sul-africano,Vinho branco,Vinho branco
1,Vinho tinto argentino,Vinho tinto,Vinho tinto
6,Vinho tinto chileno,Vinho tinto,Vinho tinto
9,Vinho tinto chileno,Vinho tinto,Vinho tinto
10,Vinho tinto argentino,Vinho tinto,Vinho tinto
...,...,...,...
613,Vinho tinto argentino,Vinho tinto,Vinho tinto
614,Espumante Rosé brasileiro,Espumante Rosé,Espumante Rosé
619,Vinho tinto italiano,Vinho tinto,Vinho tinto
621,Vinho tinto chileno,Vinho tinto,Vinho tinto


In [123]:
db["product_type"].unique()

array(['Vinho branco sul-africano', 'Vinho tinto argentino',
       'Vinho tinto chileno', 'Vinho tinto português',
       'Vinho tinto europeu', 'Vinho rosé europeu', 'Vinho rosé chileno',
       'Vinho tinto italiano', 'Vinho tinto espanhol',
       'Vinho rosé argentino', 'Vinho branco chileno',
       'Espumante Branco espanhol', 'Vinho rosé espanhol',
       'Vinho branco argentino', 'Vinho branco italiano',
       'Vinho branco português', 'Espumante Branco brasileiro',
       'Vinho branco espanhol', 'Vinho tinto francês',
       'Vinho rosé sul-africano', 'Vinho rosé português',
       'Vinho rosé italiano', 'Espumante Branco francês',
       'Espumante Rosé francês', 'Vinho branco francês',
       'Espumante Rosé espanhol', 'Espumante Branco argentino',
       'Espumante Rosé brasileiro', 'Vinho rosé francês',
       'Vinho tinto sul-africano', 'Vinho branco brasileiro'],
      dtype=object)

A variável product_type é uma informação mais detalhada das outras duas variáveis

In [124]:
db['wine_variety_equal_tech_sheet'] = db['wine_variety']==db['technical_sheet_wine_type']
db['wine_grapes_equal_tech_sheet'].value_counts()

C:\Users\victo\AppData\Local\Temp\ipykernel_30328\1868718011.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db['wine_variety_equal_tech_sheet'] = db['wine_variety']==db['technical_sheet_wine_type']


wine_grapes_equal_tech_sheet
True    281
Name: count, dtype: int64

Também podemos desconsiderar uma das duas variáveis dado que elas são iguais.

## Features: wine_region e technical_sheet_country

### Resultado: Podemos desconsiderar uma das duas variáveis 'wine_region' ou'technical_sheet_country' dado que são iguais.

In [172]:
db[['wine_region', 'technical_sheet_region', 'technical_sheet_country']].head(5)

,wine_region,technical_sheet_region,technical_sheet_country
0,África do Sul,Western Cape,África do Sul
1,Argentina,Mendoza,Argentina
6,Chile,Valle del Maule,Chile
9,Chile,Valle Central,Chile
10,Argentina,Mendoza,Argentina


In [137]:
db['wine_region'] = db['wine_region'].str.replace("•", "")

C:\Users\victo\AppData\Local\Temp\ipykernel_30328\835970862.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db['wine_region'] = db['wine_region'].str.replace("•", "")


In [146]:
db[['wine_region','technical_sheet_country']].value_counts()

wine_region    technical_sheet_country
Argentina      Argentina                  70
Chile          Chile                      65
Espanha        Espanha                    47
Itália         Itália                     34
Portugal       Portugal                   20
França         França                     15
Europeu        Europeu                    14
Brasil         Brasil                     11
África do Sul  África do Sul               5
Name: count, dtype: int64

In [141]:
db['wine_region_equal_tech_sheet'] = db['wine_region']==db['technical_sheet_country']
db['wine_region_equal_tech_sheet'].value_counts()

C:\Users\victo\AppData\Local\Temp\ipykernel_30328\3660880693.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db['wine_region_equal_tech_sheet'] = db['wine_region']==db['technical_sheet_country']


wine_region_equal_tech_sheet
True    281
Name: count, dtype: int64

In [147]:
db[['wine_region','technical_sheet_country']].head(5)

,wine_region,technical_sheet_country
0,África do Sul,África do Sul
1,Argentina,Argentina
6,Chile,Chile
9,Chile,Chile
10,Argentina,Argentina


Podemos desconsiderar uma das duas variáveis 'wine_region' ou'technical_sheet_country' dado que são iguais.

## Features: fruit_tasting, tannin_tasting, sugar_tasting e acidity_tasting


### Resultado: Como todos os itens que não possuem avaliação são os mesmos, a melhor alternativa é tirá-los para não serem um problema no modelo de machine learning.

In [163]:
db.loc[db['fruit_tasting'].isna()==True]['id'].tolist()

[238, 261, 460, 499, 559, 557, 42]

In [164]:
db.loc[db['tannin_tasting'].isna()==True]['id'].tolist()

[238, 261, 460, 499, 559, 557, 42]

In [165]:
db.loc[db['sugar_tasting'].isna()==True]['id'].tolist()

[238, 261, 460, 499, 559, 557, 42]

In [166]:
db.loc[db['acidity_tasting'].isna()==True]['id'].tolist()

[238, 261, 460, 499, 559, 557, 42]

Como todos os itens que não possuem avaliação são os mesmos, a melhor alternativa é tirá-los para não serem um problema no modelo de machine learning.

In [170]:
db = db.loc[db['acidity_tasting'].isna()==False]

In [171]:
db['id'].count()

np.int64(274)

## Feature: Próxima